# Kymograph maker
This notebook will find competitive cellular "events" in the simplest definition (i.e. a loser cell apoptosis) and return information about the spatiotemporal distrubition of counterpart competitive events (i.e. winner cell mitosis) in the form of a kymograph.

kymograph:
/ˈkʌɪmə(ʊ)ɡrɑːf/
_noun_ 
a graphical representation of spatial positions (of wild-type cells or mitoses) over time in which a spatial axis represents time, focused around an apoptotic event.

Contents:

- Load modules
- Set experiment data path
- Load tracking data
- Isolate tracks of interest (target track)
- Iterate over tracks, generating and saving kymographs

To-do:

- [x] Rewrite this to iterate over many target tracks
- [ ] Think about saving out as something other than .npy

In [1]:
import napari
import btrack
import numpy as np
from skimage.io import imread
import os
print("Napari version no.:", napari.__version__)
print("btrack version no.:", btrack.__version__)
from btrack.utils import import_HDF, import_JSON, tracks_to_napari
from tqdm.notebook import tnrange, tqdm
import matplotlib.pyplot as plt

def find_apoptosis_time(target_track, index): ### if index is set to True then the index of the apoptotic time (wrt target_track) is returned
    for i, j in enumerate(target_track.label):
        if j == 'APOPTOSIS' and target_track.label[i+1] == 'APOPTOSIS' and target_track.label[i+2] == 'APOPTOSIS': # and target_track.label[i+3] =='APOPTOSIS' and target_track.label[i+4] =='APOPTOSIS':
            apop_index = i
            break
    apop_time = target_track.t[apop_index]
    if index == True: 
        return apop_index
    else: 
        return apop_time

Napari version no.: 0.4.0
btrack version no.: 0.4.0


### Set experiment data path 

In [2]:
# print("Input experiment number")
# experiment_no = input()
# root_path = os.path.join('/home/nathan/data/kraken/h2b/giulia/', experiment_no)
root_path = '/home/nathan/data/kraken/h2b/giulia/GV0807'  ## this overwrites input option for ease 
gfp_path = os.path.join(root_path, 'Pos3/stacks/gfp.tif')
rfp_path = os.path.join(root_path, 'Pos3/stacks/rfp.tif')
bf_path = os.path.join(root_path, 'Pos3/stacks/bf.tif')
tracks_path = os.path.join(root_path, 'Pos3/Pos3_aligned/HDF/segmented.hdf5')

### Load tracking data

In [3]:
with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_1") as hdf:
    wt_tracks = hdf.tracks
with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_2") as hdf:
    scr_tracks = hdf.tracks
print("Tracks loaded")

[INFO][2021/01/03 05:12:53 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2021/01/03 05:12:53 PM] Loading tracks/obj_type_1
[INFO][2021/01/03 05:12:56 PM] Loading objects/obj_type_1 (408973, 5) (388394 filtered: area>=100)
[INFO][2021/01/03 05:13:00 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5
[INFO][2021/01/03 05:13:00 PM] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2021/01/03 05:13:00 PM] Loading tracks/obj_type_2
[INFO][2021/01/03 05:13:00 PM] Loading objects/obj_type_2 (12115, 5) (8894 filtered: area>=100)
[INFO][2021/01/03 05:13:00 PM] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5


Tracks loaded


In [4]:
scr_apops = [scr_track for scr_track in scr_tracks if scr_track.fate.name == 'APOPTOSIS']

### Set kymograph parameters

In [5]:
radius = 400
delta_t = 400
num_radial_bins = 20
radial_bin = radius / num_radial_bins
num_temporal_bins = 20
temporal_bin = delta_t / num_temporal_bins
print("size of temporal and radial bins:",temporal_bin,",", radial_bin)

size of temporal and radial bins: 20.0 , 20.0


### running on scr_ID 17 first to check against other analysis


In [ ]:
max_N_wt = 100
for scr_track in tqdm(scr_apops):
    target_track = scr_track 
    apop_index = find_apoptosis_time(target_track, index = True)
    apop_time = find_apoptosis_time(target_track, index = False) 
    ## instead of going over all tracks, could go over all tracks that are definitely in_frame (using func) or in zone 
    ## set empty data variables to store 
    num_wt, num_wt_mito = np.zeros((num_radial_bins, num_temporal_bins)), np.zeros((num_radial_bins, num_temporal_bins))
    wt_IDs, wt_mito_IDs = np.zeros((num_radial_bins, num_temporal_bins, max_N_wt)), np.zeros((num_radial_bins, num_temporal_bins, max_N_wt))
    ## load one wt_track
    for j, wt_track in enumerate(tqdm(wt_tracks)):
        print("ID:", wt_track.ID)
        ## load first distance/radial bin
        for n in range(num_radial_bins):
            #print("radial", range(int(radial_bin * n), int(radial_bin * (n+1))))
            ## load first temporal bin, over negative and positive range
            for l, m in enumerate(range(int(-num_temporal_bins/2), int(num_temporal_bins/2))): ## l introduced to iterate through pos integers for data storage
                #print("time",range(int(apop_time+temporal_bin*m),int(apop_time+temporal_bin*(m+1))))
                ## load first timepoint of wt_track
                for i in range(len(wt_track)): 
                    ## calculate if within euclidean distance: if min of radial distance bin sqrd < (wt_track.x (at first time point) - target_track.x (at apop_time)) both squared + corresponding for yboth squared < max_dist squared
                    if ((radial_bin * n)**2) < (wt_track.x[i] - target_track.x[apop_index])**2 + (wt_track.y[i] - target_track.y[apop_index])**2 < ((radial_bin * (n+1))**2):
                        ## if FIRST time point is in temporal bin (centered around apop_time)
                        if wt_track.t[i] in (range(int(apop_time+temporal_bin*m),int(apop_time+temporal_bin*(m+1)))):
                            num_wt[n,l] += 1 #str(wt_track.ID)
                            print("wt_track", wt_track.ID)
                            print("radial", range(int(radial_bin * n), int(radial_bin * (n+1))))
                            print("time",range(int(apop_time+temporal_bin*m),int(apop_time+temporal_bin*(m+1))))

                            ## storing ID in 3D ndarray
                            for p in range(len(wt_IDs[n,l,:])):
                                if wt_IDs[n,l,p] == 0:
                                    wt_IDs[n,l,p] = wt_track.ID
                                    break                        

                            #print("WT ID:", wt_track.ID, "time:", wt_track.t[i], "radial bin:", (n*radial_bin),"-", ((n+1)*radial_bin), "temporal bin:", (temporal_bin * m),"-", (temporal_bin * (m+1)),"i,m,n:", i,m,n)
                            if wt_track.fate.name == "DIVIDE" and "'METAPHASE', 'METAPHASE'," in str(wt_track.label[int(apop_time+(temporal_bin*m))-wt_track.t[0]:int(apop_time+(temporal_bin*(m+1)))-wt_track.t[0]]): ##explainer: this double condition states that if wt_track ends in mitosis but also has THREE (?) sequential metaphase classifications within the time window of that temporal bin then the condition is met
                                num_wt_mito[n,l] += 1 #str(wt_track.ID)
                                print("wt_track mitosis", wt_track.ID)
                                print("radial", range(int(radial_bin * n), int(radial_bin * (n+1))))
                                print("time",range(int(apop_time+temporal_bin*m),int(apop_time+temporal_bin*(m+1))))


                                ## storing ID in 3D ndarray
                                for p in range(len(wt_mito_IDs[n,l,:])):
                                    if wt_mito_IDs[n,l,p] == 0:
                                        wt_IDs[n,l,p] = wt_track.ID
                                        break   


                                #print("MITO ID:", wt\_track.ID, "time:", wt_track.t[i], "radial bin:", (n*radial_bin),"-", ((n+1)*radial_bin), "temporal bin:", (temporal_bin * m),"-", (temporal_bin * (m+1)),"i,m,n:", i,m,n)
                            break ## if track is in bin, break loop iterating over all track positions to skip to next bin

    scr_ID_fn = 'Scr_'+str(target_track.ID)
    fn = os.path.join(root_path, 'analysis/raw_numbers/', scr_ID_fn)

    num_wt_fn = fn + '_num_wt'
    num_wt_mito_fn = fn + 'num_wt_mito'
    wt_IDs_fn = fn + 'wt_IDs'
    wt_mito_IDs_fn = fn + 'wt_mito_IDs'

    np.save(num_wt_fn, num_wt)
    np.save(num_wt_mito_fn, num_wt_mito)
    np.save(wt_IDs_fn, wt_IDs)
    np.save(wt_mito_IDs_fn, wt_mito_IDs)

ID: 36
ID: 25
ID: 26
ID: 27
ID: 28
ID: 29
ID: 30
ID: 31
ID: 32
ID: 33
wt_track 33
radial range(380, 400)
time range(750, 770)
wt_track 33
radial range(380, 400)
time range(770, 790)
ID: 34
ID: 35
ID: 2
ID: 37
ID: 38
ID: 39
ID: 40
ID: 41
ID: 42
wt_track 42
radial range(160, 180)
time range(710, 730)
wt_track 42
radial range(160, 180)
time range(730, 750)
wt_track 42
radial range(180, 200)
time range(730, 750)
wt_track 42
radial range(180, 200)
time range(750, 770)
wt_track 42
radial range(200, 220)
time range(750, 770)
wt_track 42
radial range(200, 220)
time range(930, 950)
wt_track 42
radial range(220, 240)
time range(750, 770)
wt_track 42
radial range(220, 240)
time range(790, 810)
wt_track 42
radial range(220, 240)
time range(810, 830)
wt_track 42
radial range(220, 240)
time range(830, 850)
wt_track 42
radial range(220, 240)
time range(850, 870)
wt_track mitosis 42
radial range(220, 240)
time range(850, 870)
wt_track 42
radial range(220, 240)
time range(870, 890)
wt_track 42
radial r

wt_track 295
radial range(340, 360)
time range(910, 930)
wt_track 295
radial range(340, 360)
time range(930, 950)
ID: 296
ID: 297
ID: 298
ID: 300
ID: 301
ID: 304
ID: 305
ID: 306
ID: 307
ID: 308
ID: 309
ID: 310
ID: 311
ID: 312
ID: 313
ID: 315
ID: 316
ID: 317
ID: 318
ID: 319
ID: 320
ID: 322
ID: 323
ID: 324
ID: 326
ID: 327
ID: 328
ID: 329
ID: 330
ID: 331
ID: 332
ID: 333
wt_track 333
radial range(380, 400)
time range(710, 730)
wt_track 333
radial range(380, 400)
time range(730, 750)
wt_track 333
radial range(380, 400)
time range(750, 770)
ID: 334
ID: 335
ID: 336
ID: 337
ID: 338
ID: 340
ID: 339
ID: 341
ID: 346
ID: 345
ID: 343
ID: 342
ID: 344
ID: 347
ID: 348
ID: 349
ID: 350
ID: 351
ID: 352
ID: 353
ID: 354
ID: 355
ID: 356
ID: 357
ID: 362
ID: 361
ID: 360
ID: 359
ID: 358
ID: 363
ID: 364
ID: 365
ID: 366
ID: 367
ID: 368
ID: 369
ID: 371
ID: 372
ID: 374
ID: 375
ID: 377
ID: 378
ID: 379
ID: 380
wt_track 380
radial range(200, 220)
time range(790, 810)
wt_track 380
radial range(200, 220)
time range(810

wt_track 421
radial range(240, 260)
time range(750, 770)
wt_track 421
radial range(240, 260)
time range(770, 790)
ID: 423
ID: 425
ID: 424
ID: 422
ID: 426
ID: 427
ID: 428
wt_track 428
radial range(260, 280)
time range(1050, 1070)
wt_track 428
radial range(260, 280)
time range(1070, 1090)
wt_track 428
radial range(260, 280)
time range(1090, 1110)
wt_track 428
radial range(280, 300)
time range(710, 730)
wt_track 428
radial range(280, 300)
time range(790, 810)
wt_track 428
radial range(280, 300)
time range(810, 830)
wt_track 428
radial range(280, 300)
time range(910, 930)
wt_track 428
radial range(280, 300)
time range(930, 950)
wt_track 428
radial range(280, 300)
time range(950, 970)
wt_track 428
radial range(280, 300)
time range(970, 990)
wt_track 428
radial range(280, 300)
time range(990, 1010)
wt_track 428
radial range(280, 300)
time range(1010, 1030)
wt_track 428
radial range(280, 300)
time range(1030, 1050)
wt_track 428
radial range(280, 300)
time range(1050, 1070)
wt_track 428
radial

ID: 529
wt_track 529
radial range(300, 320)
time range(710, 730)
wt_track 529
radial range(300, 320)
time range(730, 750)
wt_track 529
radial range(300, 320)
time range(750, 770)
wt_track 529
radial range(300, 320)
time range(770, 790)
wt_track 529
radial range(320, 340)
time range(750, 770)
ID: 531
ID: 532
ID: 533
ID: 536
ID: 535
ID: 534
wt_track 534
radial range(160, 180)
time range(1050, 1070)
wt_track 534
radial range(160, 180)
time range(1070, 1090)
wt_track 534
radial range(160, 180)
time range(1090, 1110)
wt_track 534
radial range(180, 200)
time range(770, 790)
wt_track 534
radial range(180, 200)
time range(790, 810)
wt_track 534
radial range(180, 200)
time range(810, 830)
wt_track 534
radial range(180, 200)
time range(870, 890)
wt_track 534
radial range(180, 200)
time range(890, 910)
wt_track 534
radial range(180, 200)
time range(910, 930)
wt_track 534
radial range(180, 200)
time range(990, 1010)
wt_track 534
radial range(180, 200)
time range(1010, 1030)
wt_track 534
radial ran

wt_track 555
radial range(360, 380)
time range(810, 830)
wt_track 555
radial range(380, 400)
time range(810, 830)
wt_track 555
radial range(380, 400)
time range(830, 850)
wt_track 555
radial range(380, 400)
time range(850, 870)
wt_track 555
radial range(380, 400)
time range(950, 970)
wt_track 555
radial range(380, 400)
time range(970, 990)
ID: 558
ID: 562
ID: 561
wt_track 561
radial range(280, 300)
time range(910, 930)
wt_track 561
radial range(280, 300)
time range(930, 950)
wt_track 561
radial range(300, 320)
time range(710, 730)
wt_track 561
radial range(300, 320)
time range(730, 750)
wt_track 561
radial range(300, 320)
time range(830, 850)
wt_track 561
radial range(300, 320)
time range(910, 930)
wt_track 561
radial range(300, 320)
time range(930, 950)
wt_track 561
radial range(300, 320)
time range(950, 970)
wt_track 561
radial range(300, 320)
time range(1090, 1110)
wt_track 561
radial range(320, 340)
time range(710, 730)
wt_track 561
radial range(320, 340)
time range(730, 750)
wt_tr

...

wt_track 751
radial range(360, 380)
time range(665, 685)
wt_track 751
radial range(360, 380)
time range(685, 705)
wt_track 751
radial range(380, 400)
time range(645, 665)
ID: 752
wt_track 752
radial range(340, 360)
time range(665, 685)
wt_track 752
radial range(360, 380)
time range(645, 665)
wt_track 752
radial range(360, 380)
time range(665, 685)
wt_track 752
radial range(360, 380)
time range(685, 705)
wt_track 752
radial range(360, 380)
time range(705, 725)
ID: 753
wt_track 753
radial range(260, 280)
time range(645, 665)
wt_track 753
radial range(260, 280)
time range(685, 705)
wt_track 753
radial range(260, 280)
time range(705, 725)
wt_track 753
radial range(280, 300)
time range(645, 665)
wt_track 753
radial range(280, 300)
time range(665, 685)
wt_track 753
radial range(280, 300)
time range(685, 705)
wt_track 753
radial range(300, 320)
time range(645, 665)
ID: 754
ID: 755
ID: 756
ID: 757
wt_track 757
radial range(300, 320)
time range(645, 665)
wt_track 757
radial range(300, 320)
time

wt_track 823
radial range(180, 200)
time range(685, 705)
wt_track 823
radial range(180, 200)
time range(705, 725)
wt_track 823
radial range(200, 220)
time range(685, 705)
wt_track 823
radial range(200, 220)
time range(705, 725)
ID: 824
ID: 825
wt_track 825
radial range(60, 80)
time range(665, 685)
wt_track 825
radial range(60, 80)
time range(685, 705)
wt_track 825
radial range(60, 80)
time range(705, 725)
wt_track 825
radial range(80, 100)
time range(685, 705)
wt_track 825
radial range(80, 100)
time range(705, 725)
ID: 826
wt_track 826
radial range(60, 80)
time range(665, 685)
wt_track 826
radial range(60, 80)
time range(685, 705)
wt_track 826
radial range(60, 80)
time range(705, 725)
ID: 829
ID: 828
ID: 831
ID: 833
ID: 832
ID: 834
wt_track 834
radial range(180, 200)
time range(665, 685)
wt_track 834
radial range(180, 200)
time range(685, 705)
wt_track 834
radial range(200, 220)
time range(685, 705)
ID: 835
ID: 836
wt_track 836
radial range(280, 300)
time range(665, 685)
wt_track 836
r

wt_track 933
radial range(380, 400)
time range(705, 725)
ID: 938
ID: 937
ID: 936
wt_track 936
radial range(240, 260)
time range(705, 725)
wt_track 936
radial range(260, 280)
time range(705, 725)
wt_track 936
radial range(280, 300)
time range(705, 725)
ID: 934
ID: 935
wt_track 935
radial range(240, 260)
time range(705, 725)
wt_track 935
radial range(260, 280)
time range(705, 725)
ID: 939
ID: 940
ID: 941
ID: 943
ID: 944
wt_track 944
radial range(240, 260)
time range(705, 725)
ID: 945
ID: 946
ID: 947
ID: 948
ID: 949
ID: 950
ID: 951
ID: 952
ID: 953
ID: 954
ID: 955
ID: 956
wt_track 956
radial range(280, 300)
time range(705, 725)
ID: 958
ID: 959
wt_track 959
radial range(320, 340)
time range(705, 725)
wt_track 959
radial range(340, 360)
time range(705, 725)
ID: 960
wt_track 960
radial range(340, 360)
time range(705, 725)
ID: 961
wt_track 961
radial range(260, 280)
time range(705, 725)
wt_track 961
radial range(280, 300)
time range(705, 725)
ID: 962
ID: 963
ID: 964
ID: 966
ID: 965
wt_track 96

ID: 1837
ID: 1838
ID: 1843
ID: 1846
ID: 1845
ID: 1841
ID: 1840
ID: 1839
ID: 1847
ID: 1848
ID: 1849
ID: 1850
ID: 1851
ID: 1852
ID: 1853
ID: 1861
ID: 1863
ID: 1862
ID: 1860
ID: 1857
ID: 1856
ID: 1855
ID: 1858
ID: 1865
ID: 1866
ID: 1867
ID: 1868
ID: 1869
ID: 1870
ID: 1871
ID: 1872
ID: 1873
ID: 1878
ID: 1880
ID: 1877
ID: 1876
ID: 1875
ID: 1881
ID: 1882
ID: 1883
ID: 1884
ID: 1886
ID: 1887
ID: 1888
ID: 1901
ID: 1903
ID: 1905
ID: 1906
ID: 1907
ID: 1908
ID: 1909
ID: 1895
ID: 1899
ID: 1897
ID: 1896
ID: 1894
ID: 1893
ID: 1891
ID: 1890
ID: 1900
ID: 1917
ID: 1918
ID: 1916
ID: 1914
ID: 1913
ID: 1912
ID: 1910
ID: 1922
ID: 1923
ID: 1924
ID: 1925
ID: 1926
ID: 1927
ID: 1928
ID: 1929
ID: 1930
ID: 1931
ID: 1936
ID: 1938
ID: 1939
ID: 1937
ID: 1934
ID: 1933
ID: 1932
ID: 1935
ID: 1947
ID: 1950
ID: 1949
ID: 1948
ID: 1946
ID: 1945
ID: 1942
ID: 1941
ID: 1940
ID: 1952
ID: 1961
ID: 1960
ID: 1959
ID: 1958
ID: 1957
ID: 1956
ID: 1955
ID: 1954
ID: 1953
ID: 1969
ID: 1968
ID: 1967
ID: 1966
ID: 1964
ID: 1965
ID: 1962
I

ID: 2869
ID: 2870
ID: 2871
ID: 2872
ID: 2874
ID: 2875
ID: 2877
ID: 2878
ID: 2879
ID: 2880
ID: 2881
ID: 2882
ID: 2885
ID: 2886
ID: 2888
ID: 2891
ID: 2896
ID: 2895
ID: 2894
ID: 2892
ID: 2890
ID: 2889
ID: 2898
ID: 2899
ID: 2900
ID: 2902
ID: 2903
ID: 2905
ID: 2908
ID: 2912
ID: 2911
ID: 2910
ID: 2909
ID: 2907
ID: 2906
ID: 2914
ID: 2925
ID: 2924
ID: 2923
ID: 2922
ID: 2921
ID: 2919
ID: 2918
ID: 2917
ID: 2916
ID: 2915
ID: 2930
ID: 2937
ID: 2936
ID: 2934
ID: 2933
ID: 2931
ID: 2927
ID: 2929
ID: 2928
ID: 2932
ID: 2950
ID: 2951
ID: 2949
ID: 2947
ID: 2946
ID: 2944
ID: 2943
ID: 2942
ID: 2941
ID: 2938
ID: 2945
ID: 2965
ID: 2966
ID: 2967
ID: 2968
ID: 2969
ID: 2970
ID: 2971
ID: 2972
ID: 2973
ID: 2974
ID: 2975
ID: 2963
ID: 2964
ID: 2952
ID: 2953
ID: 2954
ID: 2955
ID: 2956
ID: 2957
ID: 2959
ID: 2958
ID: 2962
ID: 2960
ID: 2961
ID: 2976
ID: 2977
ID: 2978
ID: 2979
ID: 2980
ID: 2981
ID: 2982
ID: 2983
ID: 2984
ID: 2985
ID: 2991
ID: 2990
ID: 2989
ID: 2988
ID: 2987
ID: 2986
ID: 3000
ID: 2999
ID: 2998
ID: 2997
I

In [ ]:
### generate labels for axis
xlocs = range(0, num_temporal_bins, 2) ## step of 2 to avoid crowding
xlabels = []
for m in range(int(-num_temporal_bins/2), int(num_temporal_bins/2), 2):
    xlabels.append((num_temporal_bins/2)*m)
print("xlabels:",xlabels, type(xlabels))
ylocs = range(0, num_radial_bins, 2) ## step of 2 to avoid crowding
ylabels = []
for m in range(0, num_radial_bins, 2):
    ylabels.append((num_temporal_bins/2)*m)
print("ylabels:", ylabels, type(ylabels))

In [ ]:
for scr_track in scr_tracks:
    
    ## gross wt
    title = "Scr_" + str(scr_track.ID) + "_num_wt.png"
    plt.title('No. of wild-type cells')
    plt.xlabel("time bins")
    plt.ylabel("radial bins")
    plt.yticks(ylocs, ylabels)
    plt.xticks(xlocs, xlabels, rotation='vertical')
    plt.imshow(num_wt)
    plt.colorbar()
    plt.savefig(os.path.join(root_path, title), bbox_inches = 'tight')
    
    ## mitoses
    title = "Scr_" + str(scr_track.ID) + "_num_wt_mitoses.png"
    plt.title('mitosis kymograph')
    plt.xlabel("time bins")
    plt.ylabel("radial bins")
    plt.ylim(0, num_radial_bins-0.5)
    plt.yticks(ylocs, ylabels)
    plt.xticks(xlocs, xlabels, rotation='vertical')
    plt.imshow(num_wt_mito)
    plt.colorbar()
    plt.savefig(os.path.join(root_path, title), bbox_inches = 'tight')

    ## probability
    title = "Scr_" + str(scr_track.ID) + "_prob_wt_mitoses.png"
    plt.title('Probability of mitosis kymograph')
    plt.xlabel("time bins")
    plt.ylabel("radial bins")
    plt.ylim(0, num_radial_bins-0.5)
    plt.yticks(ylocs, ylabels)
    plt.xticks(xlocs, xlabels, rotation='vertical')
    prob_mito = num_wt_mito / num_wt
    plt.imshow(prob_mito)
    plt.colorbar()
    plt.savefig(os.path.join(root_path, title), bbox_inches = 'tight')


# need to write smthn to save out kymographs with unique titles